<a href="https://colab.research.google.com/github/rlawogjs96/deeplearningnlp_starters/blob/main/TF_IDF(Term_Frequency_Inverse_Document_Frequency).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. TF-IDF (단어 빈도-역 문서 빈도)
단어의 빈도와 역 문서 빈도 (문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법이다. DTM을 만든 후, TF-IDF 가중치를 부여한다. 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업. 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있다.

* tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수. 
* df(t) : 특정 단어 t가 등장한 문서의 수. 
* idf(d,t) : df(t)에 반비례하는 수. 

TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단한다. TF-IDF값이 낮으면 중요도가 낮은 것이며, Tf-IDF값이 크면 중요도가 큰것이다. 

In [2]:
import pandas as pd # For Dataframe
from math import log 

docs = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('size of 단어장: ', len(vocab)) 
print(vocab)

size of 단어장:  9
['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [3]:
# 총 문서의 수 
N = len(docs)

def tf(t,d): 
  return d.count(t) 

def idf(t): 
  df = 0 
  for doc in docs: 
    df += t in doc 
  return log(N/(df+1)) 

def tfidf(t,d): 
  return tf(t,d) * idf(t) 

In [7]:
result = []

# 각 문서에 대해서 아래 연산을 반복 
for i in range(N): 
  result.append([])
  d = docs[i]
  for j in range(len(vocab)): 
    t = vocab[j]
    result[-1].append(tf(t,d)) 

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [6]:
result = []
for j in range(len(vocab)): 
  t = vocab[j]
  result.append(idf(t)) 

idf_ = pd.DataFrame(result, index=vocab,columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [8]:
result = []
for i in range(N): 
  result.append([])
  d = docs[i]
  for j in range(len(vocab)): 
    t = vocab[j]
    result[-1].append(tfidf(t,d)) 

tfidf_ = pd.DataFrame(result, columns = vocab) 
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


#### 2. scikitlearn 을 이용한 DTM과 TF-IDF

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do',
]

vector = CountVectorizer() 

# 코퍼스로부터 각 단어의 빈도수를 기록 
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력 
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


DTM에서 각 단어의 인덱스가 어떻게 부여되었는지를 확인하기 위해 인덱스를 확인. 0의 인덱스 do는 세번째 문서에서만 등장했기 때문에 세번째 행에서만 1의 값을 가진다. 두번째 열의 경우에는 1의 인덱스를 가진 know이다. know는 첫번째 문서에서만 등장했으므로 첫번째 행에서만 1의 값을 가진다. 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you', 
    'what should I do',
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
